In [8]:
#数据的预处理
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
pd.set_option('display.max_row',1000)
#导入数据
filename = r'G:\科研\人工智能\机器学习\机器学习\data.csv'
data=pd.read_csv(filename,encoding='gbk')
print("data.shape:",data.shape) 

data.shape: (4754, 90)


In [9]:
#数据处理
miss_rate = data.isnull().sum() / len(data)
print("缺失率：",miss_rate.sort_values(ascending=False))
X_num = data.select_dtypes('number').copy()
X_num.fillna(X_num.mean(),inplace=True)
print("数值型特征的shape：",X_num.shape)
print(X_num.columns)
X_num.drop(['Unnamed: 0','status'],axis=1,inplace=True)
 
X_str = data.select_dtypes(exclude='number').copy()
X_str.fillna(0,inplace=True)
print("非数值型特征：",X_str.columns)
print(X_str.head())
 
X_dummy = pd.get_dummies(X_str['reg_preference_for_trad'])
X = pd.concat([X_num,X_dummy],axis=1,sort=False)
y = data['status']
#划分训练集、测试集
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=2018)

缺失率： student_feature                               0.630627
cross_consume_count_last_1_month              0.089609
apply_credibility                             0.063946
query_cash_count                              0.063946
latest_six_month_apply                        0.063946
latest_three_month_apply                      0.063946
latest_query_time                             0.063946
query_sum_count                               0.063946
latest_one_month_apply                        0.063946
query_finance_count                           0.063946
query_org_count                               0.063946
apply_score                                   0.063946
latest_query_day                              0.063946
latest_one_month_loan                         0.062474
loans_score                                   0.062474
loans_credibility_behavior                    0.062474
loans_count                                   0.062474
loans_settle_count                            0.062474
loans

In [18]:
#归一化
dataset= StandardScaler()
X_train_std = dataset.fit_transform(X_train)
X_test_std = dataset.transform(X_test)
print("f1_score:")

f1_score:


D:\ac\anaconda\envs\py36\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\ac\anaconda\envs\py36\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\ac\anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [30]:
from sklearn.metrics import accuracy_score, roc_auc_score
def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    # 准确率
    print('[准确率]', end = ' ')
    print('训练集：', '%.4f'%accuracy_score(y_train, y_train_pred), end = ' ')
    print('测试集：', '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print('[auc值]', end = ' ')
    print('训练集：', '%.4f'%roc_auc_score(y_train, y_train_proba), end = ' ')
    print('测试集：', '%.4f'%roc_auc_score(y_test, y_test_proba))


In [31]:
#逻辑回归模型
lr = LogisticRegression()
lr.fit(X_train_std,y_train)
test_lr_pred = lr.predict(X_test_std)
print("逻辑回归模型:",f1_score(y_test,test_lr_pred,average='macro'))
model_metrics(lr, X_train, X_test, y_train, y_test)

逻辑回归模型: 0.6499021443532617
[准确率] 训练集： 0.7493 测试集： 0.7484
[auc值] 训练集： 0.5000 测试集： 0.5000


In [36]:
#svm模型
lsvc = LinearSVC()
lsvc.fit(X_train_std,y_train)
test_lsvc_pred = lsvc.predict(X_test_std)
print("svm模型:",f1_score(y_test,test_lsvc_pred,average='macro'))

svm模型: 0.6268382887052388


In [21]:
#决策树模型
dt = DecisionTreeClassifier()
dt.fit(X_train_std,y_train)
test_dt_pred = dt.predict(X_test_std)
print("决策树模型:",f1_score(y_test,test_dt_pred))


决策树模型: 0.39463087248322143


In [39]:
#xgboost
from xgboost.sklearn import XGBClassifier
import warnings
warnings.filterwarnings("ignore")
xgb= XGBClassifier()
xgb.fit(X_train, y_train)
model_metrics(xgb, X_train, X_test, y_train, y_test)


[准确率] 训练集： 0.8548 测试集： 0.7793
[auc值] 训练集： 0.9171 测试集： 0.7686


In [43]:
#随机森林
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=250, max_depth=4, n_jobs=-1, oob_score=True, random_state=2018, verbose=1)
std = StandardScaler()
std.fit(X_train)
x_train_std = std.transform(X_train)
x_test_std = std.transform(X_test)
random_forest.fit(x_train_std, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.6s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=-1,
            oob_score=True, random_state=2018, verbose=1, warm_start=False)